In [155]:
# Dependencies and set up
import pandas as pd
import numpy as np
import json

import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [96]:
# Chicago Crimes - 2021
url_too = 'https://data.cityofchicago.org/resource/crimes.json'
df_too = pd.read_json(url_too)
df_too

,id,case_number,date,block,iucr,primary_type,description,location_description,arrest,domestic,...,longitude,location,:@computed_region_awaf_s7ux,:@computed_region_6mkv_f3dw,:@computed_region_vrxf_vc4k,:@computed_region_bdys_3d7i,:@computed_region_43wa_7qmu,:@computed_region_rpca_8um6,:@computed_region_d9mm_jgwp,:@computed_region_d3ds_rm58
0,12437917,JE319294,2021-07-29 23:55:00,024XX E 79TH ST,0430,BATTERY,AGGRAVATED - OTHER DANGEROUS WEAPON,APARTMENT,False,False,...,-87.565800,"{'latitude': '41.751733954', 'longitude': '-87...",43.0,21202.0,39.0,246.0,37.0,24.0,19.0,225.0
1,12437961,JE319270,2021-07-29 23:46:00,048XX S HAMLIN AVE,031A,ROBBERY,ARMED - HANDGUN,SIDEWALK,False,False,...,-87.718593,"{'latitude': '41.804940137', 'longitude': '-87...",49.0,22248.0,54.0,698.0,8.0,56.0,13.0,103.0
2,12437874,JE319259,2021-07-29 23:46:00,042XX S MICHIGAN AVE,141B,WEAPONS VIOLATION,UNLAWFUL USE - OTHER FIREARM,STREET,False,False,...,-87.623057,"{'latitude': '41.817597349', 'longitude': '-87...",12.0,4301.0,4.0,163.0,9.0,36.0,24.0,102.0
3,12438026,JE319261,2021-07-29 23:45:00,004XX N HAMLIN AVE,0810,THEFT,OVER $500,APARTMENT,False,True,...,-87.721003,"{'latitude': '41.888787538', 'longitude': '-87...",41.0,21572.0,24.0,584.0,46.0,30.0,16.0,82.0
4,12438635,JE319437,2021-07-29 23:45:00,003XX E RANDOLPH ST,0870,THEFT,POCKET-PICKING,PARK PROPERTY,False,False,...,-87.618470,"{'latitude': '41.884355847', 'longitude': '-87...",22.0,14309.0,38.0,580.0,36.0,41.0,22.0,87.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,12436285,JE317111,2021-07-28 10:42:00,005XX W HAWTHORNE PL,143A,WEAPONS VIOLATION,UNLAWFUL POSSESSION - HANDGUN,APARTMENT,True,False,...,-87.644262,"{'latitude': '41.944315717', 'longitude': '-87...",38.0,4449.0,57.0,169.0,25.0,22.0,5.0,18.0
996,12436203,JE317082,2021-07-28 10:40:00,056XX W CHICAGO AVE,0420,BATTERY,AGGRAVATED - KNIFE / CUTTING INSTRUMENT,STREET,False,False,...,-87.767129,"{'latitude': '41.894828303', 'longitude': '-87...",52.0,22216.0,26.0,672.0,7.0,32.0,25.0,70.0
997,12436357,JE317184,2021-07-28 10:35:00,009XX E 47TH ST,0495,BATTERY,AGGRAVATED OF A SENIOR CITIZEN,APARTMENT,False,False,...,-87.602469,"{'latitude': '41.809578412', 'longitude': '-87...",1.0,4301.0,5.0,461.0,10.0,10.0,24.0,113.0
998,12436257,JE317064,2021-07-28 10:33:00,033XX S ASHLAND AVE,0486,BATTERY,DOMESTIC BATTERY SIMPLE,APARTMENT,False,True,...,-87.665619,"{'latitude': '41.832869291', 'longitude': '-87...",26.0,14920.0,56.0,192.0,1.0,43.0,23.0,165.0


In [158]:
# Raw crime table
table = pd.read_csv('/Users/karlaflores/Desktop/Git/Project-03/Crimes_-_2001_to_Present.csv') 
table

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,...,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location
0,10224738,HY411648,09/05/2015 01:30:00 PM,043XX S WOOD ST,0486,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,False,True,...,12.0,61.0,08B,1165074.0,1875917.0,2015,02/10/2018 03:50:01 PM,41.815117,-87.670000,"(41.815117282, -87.669999562)"
1,10224739,HY411615,09/04/2015 11:30:00 AM,008XX N CENTRAL AVE,0870,THEFT,POCKET-PICKING,CTA BUS,False,False,...,29.0,25.0,06,1138875.0,1904869.0,2015,02/10/2018 03:50:01 PM,41.895080,-87.765400,"(41.895080471, -87.765400451)"
2,11646166,JC213529,09/01/2018 12:01:00 AM,082XX S INGLESIDE AVE,0810,THEFT,OVER $500,RESIDENCE,False,True,...,8.0,44.0,06,NaN,NaN,2018,04/06/2019 04:04:43 PM,NaN,NaN,NaN
3,10224740,HY411595,09/05/2015 12:45:00 PM,035XX W BARRY AVE,2023,NARCOTICS,POSS: HEROIN(BRN/TAN),SIDEWALK,True,False,...,35.0,21.0,18,1152037.0,1920384.0,2015,02/10/2018 03:50:01 PM,41.937406,-87.716650,"(41.937405765, -87.716649687)"
4,10224741,HY411610,09/05/2015 01:00:00 PM,0000X N LARAMIE AVE,0560,ASSAULT,SIMPLE,APARTMENT,False,True,...,28.0,25.0,08A,1141706.0,1900086.0,2015,02/10/2018 03:50:01 PM,41.881903,-87.755121,"(41.881903443, -87.755121152)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7372382,12436418,JE317243,07/27/2021 08:41:00 PM,007XX S LOOMIS ST,1130,DECEPTIVE PRACTICE,FRAUD OR CONFIDENCE GAME,APARTMENT,False,False,...,28.0,28.0,11,1167165.0,1896772.0,2021,08/03/2021 04:47:34 PM,41.872301,-87.661731,"(41.872300825, -87.661731237)"
7372383,12435900,JE316680,07/27/2021 10:45:00 PM,006XX N LA SALLE DR,0820,THEFT,$500 AND UNDER,GAS STATION,False,False,...,42.0,8.0,06,1175023.0,1904549.0,2021,08/03/2021 04:47:34 PM,41.893469,-87.632648,"(41.893469034, -87.632648229)"
7372384,12438154,JE319557,07/12/2021 09:00:00 AM,044XX W ADAMS ST,1320,CRIMINAL DAMAGE,TO VEHICLE,STREET,False,False,...,28.0,26.0,14,1146898.0,1898645.0,2021,08/03/2021 04:45:05 PM,41.877852,-87.736093,"(41.877851597, -87.736092822)"
7372385,12435637,JE316502,07/27/2021 05:01:00 PM,033XX W VAN BUREN ST,2024,NARCOTICS,POSSESS - HEROIN (WHITE),ALLEY,True,False,...,28.0,27.0,18,1153961.0,1897823.0,2021,08/03/2021 04:47:34 PM,41.875458,-87.710181,"(41.875458145, -87.710180976)"


In [136]:
# Groupby Primary type
df = table.groupby(['Primary Type']).count()['Description']
df

Primary Type
ARSON                                  12452
ASSAULT                               470968
BATTERY                              1351614
BURGLARY                              411063
CONCEALED CARRY LICENSE VIOLATION        767
CRIM SEXUAL ASSAULT                    27866
CRIMINAL DAMAGE                       840295
CRIMINAL SEXUAL ASSAULT                 3495
CRIMINAL TRESPASS                     207053
DECEPTIVE PRACTICE                    315360
DOMESTIC VIOLENCE                          1
GAMBLING                               14602
HOMICIDE                               11275
HUMAN TRAFFICKING                         73
INTERFERENCE WITH PUBLIC OFFICER       17679
INTIMIDATION                            4351
KIDNAPPING                              7032
LIQUOR LAW VIOLATION                   14529
MOTOR VEHICLE THEFT                   339751
NARCOTICS                             738637
NON - CRIMINAL                            38
NON-CRIMINAL                             1

In [137]:
# Groupby Primary type, year and arrest
df = table.groupby(['Year', 'Primary Type', 'Arrest']).count()['ID']
df

Year  Primary Type       Arrest
2001  ARSON              False       829
                         True        181
      ASSAULT            False     24234
                         True       7150
      BATTERY            False     72986
                                   ...  
2021  STALKING           True         11
      THEFT              False     18989
                         True        837
      WEAPONS VIOLATION  False      1824
                         True       3384
Name: ID, Length: 1244, dtype: int64

In [138]:
# Groupby Primary type, year and arrest and count be case number
df = table.groupby(['Year', 'Primary Type', 'Arrest']).count()['Case Number']
df

Year  Primary Type       Arrest
2001  ARSON              False       829
                         True        181
      ASSAULT            False     24234
                         True       7150
      BATTERY            False     72986
                                   ...  
2021  STALKING           True         11
      THEFT              False     18989
                         True        837
      WEAPONS VIOLATION  False      1824
                         True       3384
Name: Case Number, Length: 1244, dtype: int64

In [139]:
# Groupby Primary type and ID
df = table.groupby(['Primary Type', 'Arrest']).count()['ID']
df

Primary Type       Arrest
ARSON              False       10881
                   True         1571
ASSAULT            False      366627
                   True       104341
BATTERY            False     1048941
                              ...   
STALKING           True          611
THEFT              False     1370800
                   True       180179
WEAPONS VIOLATION  False       21570
                   True        69791
Name: ID, Length: 71, dtype: int64

In [140]:
len(df)

71

In [141]:
# Defining table type
table['Date'].dtypes

dtype('O')

In [142]:
# Crime column type
print (table.dtypes)

ID                        int64
Case Number              object
Date                     object
Block                    object
IUCR                     object
Primary Type             object
Description              object
Location Description     object
Arrest                     bool
Domestic                   bool
Beat                      int64
District                float64
Ward                    float64
Community Area          float64
FBI Code                 object
X Coordinate            float64
Y Coordinate            float64
Year                      int64
Updated On               object
Latitude                float64
Longitude               float64
Location                 object
dtype: object


In [143]:
# Dropping table
table = table.drop(columns=['Case Number', 'Location', 'Updated On', 'Block', 'Location Description', 'Community Area', 'Beat', 'District', 'Ward', 'FBI Code', 'X Coordinate', 'Y Coordinate' ])
table

,ID,Date,IUCR,Primary Type,Description,Arrest,Domestic,Year,Latitude,Longitude
0,10224738,09/05/2015 01:30:00 PM,0486,BATTERY,DOMESTIC BATTERY SIMPLE,False,True,2015,41.815117,-87.670000
1,10224739,09/04/2015 11:30:00 AM,0870,THEFT,POCKET-PICKING,False,False,2015,41.895080,-87.765400
2,11646166,09/01/2018 12:01:00 AM,0810,THEFT,OVER $500,False,True,2018,NaN,NaN
3,10224740,09/05/2015 12:45:00 PM,2023,NARCOTICS,POSS: HEROIN(BRN/TAN),True,False,2015,41.937406,-87.716650
4,10224741,09/05/2015 01:00:00 PM,0560,ASSAULT,SIMPLE,False,True,2015,41.881903,-87.755121
...,...,...,...,...,...,...,...,...,...,...
7372382,12436418,07/27/2021 08:41:00 PM,1130,DECEPTIVE PRACTICE,FRAUD OR CONFIDENCE GAME,False,False,2021,41.872301,-87.661731
7372383,12435900,07/27/2021 10:45:00 PM,0820,THEFT,$500 AND UNDER,False,False,2021,41.893469,-87.632648
7372384,12438154,07/12/2021 09:00:00 AM,1320,CRIMINAL DAMAGE,TO VEHICLE,False,False,2021,41.877852,-87.736093
7372385,12435637,07/27/2021 05:01:00 PM,2024,NARCOTICS,POSSESS - HEROIN (WHITE),True,False,2021,41.875458,-87.710181


In [144]:
# Dropping NAN from latitude
table = table[table['Latitude'].notna()]
table

,ID,Date,IUCR,Primary Type,Description,Arrest,Domestic,Year,Latitude,Longitude
0,10224738,09/05/2015 01:30:00 PM,0486,BATTERY,DOMESTIC BATTERY SIMPLE,False,True,2015,41.815117,-87.670000
1,10224739,09/04/2015 11:30:00 AM,0870,THEFT,POCKET-PICKING,False,False,2015,41.895080,-87.765400
3,10224740,09/05/2015 12:45:00 PM,2023,NARCOTICS,POSS: HEROIN(BRN/TAN),True,False,2015,41.937406,-87.716650
4,10224741,09/05/2015 01:00:00 PM,0560,ASSAULT,SIMPLE,False,True,2015,41.881903,-87.755121
5,10224742,09/05/2015 10:55:00 AM,0610,BURGLARY,FORCIBLE ENTRY,False,False,2015,41.744379,-87.658431
...,...,...,...,...,...,...,...,...,...,...
7372382,12436418,07/27/2021 08:41:00 PM,1130,DECEPTIVE PRACTICE,FRAUD OR CONFIDENCE GAME,False,False,2021,41.872301,-87.661731
7372383,12435900,07/27/2021 10:45:00 PM,0820,THEFT,$500 AND UNDER,False,False,2021,41.893469,-87.632648
7372384,12438154,07/12/2021 09:00:00 AM,1320,CRIMINAL DAMAGE,TO VEHICLE,False,False,2021,41.877852,-87.736093
7372385,12435637,07/27/2021 05:01:00 PM,2024,NARCOTICS,POSSESS - HEROIN (WHITE),True,False,2021,41.875458,-87.710181


In [145]:
# Groupby by ID
table_2 = table.groupby(['Year', 'Primary Type', 'Arrest']).count()['ID']
table_2

Year  Primary Type       Arrest
2001  ARSON              False       824
                         True        180
      ASSAULT            False     24142
                         True       7118
      BATTERY            False     72665
                                   ...  
2021  STALKING           True         11
      THEFT              False     18518
                         True        837
      WEAPONS VIOLATION  False      1824
                         True       3383
Name: ID, Length: 1239, dtype: int64

In [146]:
print (table.dtypes)

ID                int64
Date             object
IUCR             object
Primary Type     object
Description      object
Arrest             bool
Domestic           bool
Year              int64
Latitude        float64
Longitude       float64
dtype: object


In [153]:
rds_connection_string = "postgres:123@localhost:5432/Project_03"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [154]:
# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine,reflect=True)
Base

OperationalError: (psycopg2.OperationalError) FATAL:  database "Project_03" does not exist

(Background on this error at: http://sqlalche.me/e/13/e3q8)

In [152]:
engine.table_names()

OperationalError: (psycopg2.OperationalError) FATAL:  database "Project_2" does not exist

(Background on this error at: http://sqlalche.me/e/13/e3q8)